In [ ]:
#### The code is from Irene Yi-Ju Su ####
!pip3 install datasets
!pip3 install unsloth

In [ ]:
from datasets import load_dataset
dataset = load_dataset("sentence-transformers/eli5", split = "train")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from unsloth import FastLanguageModel
import numpy as np
import torch
import re

README.md:   0%|          | 0.00/2.24k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/113M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/325475 [00:00<?, ? examples/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model_name = "unsloth/Llama-3.2-1B-Instruct"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/TEST")
import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
max_seq_length = 512
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
model = FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:  11%|#1        | 115M/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
def formatting_prompts_func(examples):
    questions = examples['question']
    answers = examples['answer']
    convos = []
    for question, answer in zip(questions, answers):
        convos.append([{"role": "user", "content": question}, {"role": "assistant", "content": answer}])
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
def stream_answer(question, model=model, tokenizer=tokenizer):
    messages = [
        {"role": "user", "content": question},
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        return_tensors = "pt",
    ).to("cuda")

    # Create attention mask
    attention_mask = torch.ones_like(inputs).to("cuda")
    input_length = inputs.shape[1]
    generated_ids = model.generate(
        input_ids = inputs,
        attention_mask = attention_mask,
        max_new_tokens = 512,
        use_cache = True,

    )
    answer = tokenizer.batch_decode(generated_ids[:, input_length:], skip_special_tokens=True)[0]

    return answer

In [ ]:
import random
random.seed(42)
selected_indices = random.sample(range(len(dataset['test'])), 247)
selected_dataset = dataset['test'].select(selected_indices)

questions, answers = [], []
for i in range(247):
  print (i)
  question = selected_dataset['question'][i]
  questions.append(question)
  #answer= stream_answer(question) #uncomment to run, very long
  #answers.append(answer)

In [ ]:
#df_QA = pd.DataFrame({"questions": questions, "answers": answers})
#df_QA.to_csv("qa_pairs3.csv")

In [ ]:
class SimplifiedSummarizer:
    def __init__(self, model_name="facebook/bart-large-cnn"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    def flatten_text(self, text: str) -> str:
        text = re.sub(r'\*\*', '', text)
        # Convert numbered lists into sentences
        lines = []
        for line in text.split('\n'):
            line = line.strip()
            if not line:
                continue
            # If it's a numbered point, remove the number but keep content
            if re.match(r'^\d+\.', line):
                line = re.sub(r'^\d+\.\s*', '', line)
            lines.append(line)
        flattened = ' '.join(lines)
        flattened = re.sub(r'\s+', ' ', flattened)
        return flattened.strip()

    def generate_summary(self, text: str, max_length: int = 150) -> str:
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=1024,
            truncation=True
        )

        outputs = self.model.generate(
            **inputs,
            max_length=max_length,
            min_length=30,
            num_beams=4,
            length_penalty=2.0,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
        )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
df_QA = pd.read_csv("qa_pairs3.csv")
questions, answers = df_QA['questions'].tolist(), df_QA['answers'].tolist()
answers_simple = []
for i in range(len(questions)):
    summarizer = SimplifiedSummarizer()
    flattened = summarizer.flatten_text(answers[i])
    summary = summarizer.generate_summary(flattened)
    answers_simple.append(summary)
    print(summary)

In [ ]:
#df_QA = pd.DataFrame({"questions": questions,  "answers": answers_simple})
#df_QA.to_csv("qa_pairs3_simple.csv", index = False)

In [ ]:
df_QA = pd.read_csv("qa_pairs3_simple.csv")
questions, answers = df_QA['questions'].tolist(), df_QA['answers'].tolist()
for i in range(len(df_QA)):
  print(i, " ", df_QA["questions"][i], "Answer: ", df_QA["answers"][i])

In [ ]:
labels = []
for i in range(len(df_QA)):
  if i in [12,35,74,82,83,86,92,94,95,96,99,156,158,174]: #review by GPT-4o carefiully according to (1) relevance to the question, (2) absence of self-contradiction, (3) logical consistency, (4) fluent flow, and (5) lack of repetition
    labels.append(int(0))
  else:
    labels.append(int(1))
df_QA["labels"] = labels
df_QA.to_csv("qa_pairs3_simple.csv", index = False)


In [ ]:
df_QA

Mounted at /content/drive


,questions,answers,labels
0,"For the most part, do vitamins behave the same...","Vitamins are organic compounds, and their chem...",1
1,Are radio waves the farthest traveled thing pr...,The cosmic microwave background radiation (CMB...,1
2,How did peanut butter sandwiches go from being...,Peanuts were first introduced to the Americas ...,1
3,Why did the Netherlands not take part in the S...,The Netherlands did take part in the Scramble ...,1
4,How does temperature work in space?,temperature in space is influenced by the amo...,1
...,...,...,...
242,How did affluent Roman's make / get their money?,Affluent Romans in ancient Rome made and obtai...,1
243,What is the Loudness War and what side am I on?,The Loudness War is a conflict between major m...,1
244,Was there an empire that preceded the native N...,"The Mesoamerican civilizations, which existed ...",1
245,This article from the 1960s makes it seem that...,The 1960s saw a significant shift in family st...,1
